In [ ]:
import re
import sys
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets

## Reading Data

In [ ]:
# Data Setting
TEXT = data.Field(batch_first = True,
                  fix_length = 500,
                  tokenize=str.split,
                  pad_first=True,
                  pad_token='[PAD]',
                  unk_token='[UNK]')

LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(text_field = TEXT, 
                                             label_field = LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.8MB/s]


In [ ]:
# Data Length
print(f'Train Data Length : {len(train_data.examples)}')
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 25000
Test Data Length : 25000


In [ ]:
# Data Fields
train_data.fields

{'label': <torchtext.data.field.LabelField at 0x7f98c6c69240>,
 'text': <torchtext.data.field.Field at 0x7f98c6c69fd0>}

In [ ]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']),'\n')
print('Label : ')
print(vars(train_data.examples[1])['label'])

---- Data Sample ----
Input : 
This is definitely an appropriate update for the original, except that "party on the left is now party on the right." Like the original, this movie rails against a federal government which oversteps its bounds with regards to personal liberty. It is a warning of how tenuous our political liberties are in an era of an over-zealous, and over-powerful federal government. Kowalski serves as a metaphor for Waco and Ruby Ridge, where the US government, with the cooperation of the mainstream media, threw around words like "white supremacist" and "right wing extremists as well as trumped-up drug charges to abridge the most fundamental of its' citizens rights, with the willing acquiescence of the general populace. That message is so non-PC, I am stunned that this film could be made - at least not without bringing the Federal government via the IRS down on the makers like they did to Juanita Broderick, Katherine Prudhomme, the Western Journalism Center, and countle

## Pre-processing Data

In [ ]:
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

In [ ]:
for example in train_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
    
for example in test_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

## Making Vocab & Setting Embedding

In [ ]:
model_config = {'emb_type' : 'fasttext', 'emb_dim' : 300}

In [ ]:
# fasttext pre-trained 

# TEXT.build_vocab(train_data,
#                  min_freq = 2, 
#                  max_size = None,
#                  vectors = 'fasttext.en.300d')

# 위와 같은 옵션으로 fasttext Pre-Trained vector를 가져올 수 있으나 현재 버그가 있어서 직접 다운받아 사용하는 방법으로 변경하였습니다.
# ISSUE : https://github.com/pytorch/text/issues/634

# 아래 코드는 jupyter notebook에서만 실행됩니다. python script로 실행할 경우에는 따로 다운받아서 준비해주시기 바랍니다.
! wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec' 

--2020-06-23 07:32:57--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6597238061 (6.1G) [binary/octet-stream]
Saving to: ‘wiki.en.vec’

wiki.en.vec         100%[===================>]   6.14G  7.33MB/s    in 8m 45s  

2020-06-23 07:41:43 (12.0 MB/s) - ‘wiki.en.vec’ saved [6597238061/6597238061]



In [ ]:
# fasttext pre-trained 
from torchtext.vocab import Vectors
fasttext_vectors = Vectors('./wiki.en.vec')

TEXT.build_vocab(train_data,
                 min_freq = 2, 
                 max_size = None,
                 vectors = fasttext_vectors)

LABEL.build_vocab(train_data)

model_config['vocab_size'] = len(TEXT.vocab)

In [ ]:
# Vocabulary Info
print(f'Vocab Size : {len(TEXT.vocab)}')

print('Vocab Examples : ')
for idx, (k, v) in enumerate(TEXT.vocab.stoi.items()):
    if idx >= 10:
        break    
    print('\t', k, v)

print('---------------------------------')

# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)

Vocab Size : 51956
Vocab Examples : 
	 [UNK] 0
	 [PAD] 1
	 the 2
	 and 3
	 a 4
	 of 5
	 to 6
	 is 7
	 in 8
	 it 9
---------------------------------
Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


In [ ]:
# Check embedding vectors
TEXT.vocab.vectors.shape

torch.Size([51956, 300])

## Spliting Validation Data & Making Data Iterator

In [ ]:
# Spliting Valid set
train_data, valid_data = train_data.split(random_state = random.seed(0),
                                          split_ratio=0.8)

In [ ]:
model_config['batch_size'] = 30

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=model_config['batch_size'],
    device=device)

In [ ]:
# Check batch data
sample_for_check = next(iter(train_iterator))
print(sample_for_check)
print(sample_for_check.text)
print(sample_for_check.label)


[torchtext.data.batch.Batch of size 30]
	[.text]:[torch.cuda.LongTensor of size 30x500 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 30 (GPU 0)]
tensor([[   1,    1,    1,  ...,   43,    5,  155],
        [   1,    1,    1,  ...,   25,    6,  132],
        [   1,    1,    1,  ...,   40, 1041,   75],
        ...,
        [   1,    1,    1,  ...,    5,   65, 1258],
        [ 100,  462,   58,  ...,    3,  148,   56],
        [   1,    1,    1,  ...,  979,  719,  123]], device='cuda:0')
tensor([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.], device='cuda:0')


In [ ]:
# Check reverting data
print(' '.join([TEXT.vocab.itos[int(x)] for x in sample_for_check.text[0,:] if x not in [0,1]]))
print(LABEL.vocab.itos[int(sample_for_check.label[0])]) 

i'm a fan of both shakespeare and mst3k so i waited anxiously to see this episode i'll comment on the movie first then the mst3k episode the recipe for this movie take talented actors rich and beautiful shakespeare material and a 1 25 budget mix well then drain of all life and movement until dull and lifeless serve cold in a big plain stone cauldron movie i give 3 out of 10 because the actors at least deserve a little bit of credit okay now the mst3k episode i'll admit it the first time i saw it i fell asleep halfway through i understand that was the reaction of several other as well however when i watched it a second time i realized that there was a whole host of intelligent references and good lines i missed the first time around the trick with this episode is listen carefully it takes a couple of viewings to catch each line give it a second chance and you'll see what i mean mst3k episode 7 1 2 out of 10
neg


## Making Model

In [ ]:
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()

        if model_config['emb_type'] == 'glove' or 'fasttext':
            self.emb = nn.Embedding(model_config['vocab_size'],
                                    model_config['emb_dim'],
                                    _weight = TEXT.vocab.vectors)
        else:
            self.emb = nn.Embedding(model_config['vocab_size'],
                                    model_config['emb_dim'])
        
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type'] 

        self.RNN = nn.RNN (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.LSTM= nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.GRU = nn.GRU (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
    
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction,
                            model_config['output_dim'])
        
        self.drop = nn.Dropout(model_config['dropout'])

    def forward(self, x):
        
        emb = self.emb(x) 
        # emb : (Batch_Size, Max_Seq_Length, Emb_dim)

        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb) 
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')
        
        # output : (Batch_Size, Max_Seq_Length, Hidden_dim * num_direction) 
        # hidden : (Batch_Size, num_direction, Hidden_dim)
        
        last_output = output[:,-1,:]

        # last_output : (Batch_Size, Hidden_dim * num_direction)
        return self.fc(self.drop(last_output))

### Checking feed-forward

In [ ]:
model_config.update(dict(batch_first = True,
                         model_type = 'RNN',
                         bidirectional = True,
                         hidden_dim = 128,
                         output_dim = 1,
                         dropout = 0))

In [ ]:
model = SentenceClassification(**model_config).to(device)

In [ ]:
predictions = model.forward(sample_for_check.text).squeeze()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss().to(device)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
loss = loss_fn(predictions, sample_for_check.label)
acc = binary_accuracy(predictions, sample_for_check.label)

In [ ]:
print(predictions)
print(loss, acc)

tensor([0.1428, 0.0793, 0.2120, 0.1943, 0.1225, 0.3271, 0.0696, 0.0818, 0.0368,
        0.2749, 0.2509, 0.1662, 0.0908, 0.1364, 0.1094, 0.0711, 0.0253, 0.0784,
        0.0629, 0.1540, 0.0161, 0.0567, 0.1145, 0.0319, 0.1674, 0.1520, 0.1143,
        0.0493, 0.0863, 0.1644], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor(0.7038, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) tensor(0.5000, device='cuda:0')


## Training

In [ ]:

def train(model, iterator, optimizer, loss_fn, idx_epoch, **model_params):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    batch_size = model_params['batch_size']

    for idx, batch in enumerate(iterator):
        
        # Initializing
        optimizer.zero_grad()
        
        # Forward 
        predictions = model(batch.text).squeeze()
        loss = loss_fn(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        sys.stdout.write(
                    "\r" + f"[Train] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )

        # Backward 
        loss.backward()
        optimizer.step()
        
        # Update Epoch Performance
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator) , epoch_acc/len(iterator) 

In [ ]:
def evaluate(model, iterator, loss_fn):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # evaluation mode
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = loss_fn(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### bi-RNN

In [ ]:
model_config['model_type'] = 'RNN'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-RNN_fasttext
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.5937  Acc : 0.7333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.5855 | Train Acc : 0.6843
	 Epoch : 0 | Valid Loss : 0.6241 | Valid Acc : 0.7313
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.3864  Acc : 0.7667
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.507 | Train Acc : 0.754
	 Epoch : 1 | Valid Loss : 0.5609 | Valid Acc : 0.719
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.5854  Acc : 0.7
	 Epoch : 2 | Train Loss : 0.38 | Train Acc : 0.8347
	 Epoch : 2 | Valid Loss : 0.6707 | Valid Acc : 0.5851
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.3719  Acc : 0.8333
	 Epoch : 3 | Train Loss : 0.4988 | Train Acc : 0.7408
	 Epoch : 3 | Valid Loss : 0.6833 | Valid Acc : 0.639
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.7006  Acc : 0.6333
	 Epoch : 4 | Train Loss : 0.3737 | Train Acc : 0.8292
	 Epoch : 4 | Valid

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.5666 | Test Acc : 0.7163


### bi-LSTM

In [ ]:
model_config['model_type'] = 'LSTM'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-LSTM_fasttext
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.9103  Acc : 0.3333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.5922 | Train Acc : 0.6895
	 Epoch : 0 | Valid Loss : 0.6722 | Valid Acc : 0.5856
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.436  Acc : 0.8
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.3893 | Train Acc : 0.8246
	 Epoch : 1 | Valid Loss : 0.3152 | Valid Acc : 0.8736
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.08638  Acc : 0.9333
	 Epoch : 2 | Train Loss : 0.1778 | Train Acc : 0.9333
	 Epoch : 2 | Valid Loss : 0.3201 | Valid Acc : 0.8787
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.00928  Acc : 1.0
	 Epoch : 3 | Train Loss : 0.07141 | Train Acc : 0.9772
	 Epoch : 3 | Valid Loss : 0.3808 | Valid Acc : 0.8746
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.07977  Acc : 0.9667
	 Epoch : 4 | Train Loss : 0.0275 | Train Acc : 0.9925
	 Epoch : 4 

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.321 | Test Acc : 0.8687


### bi-GRU

In [ ]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-GRU_fasttext
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.4283  Acc : 0.8667
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.4061 | Train Acc : 0.8075
	 Epoch : 0 | Valid Loss : 0.2966 | Valid Acc : 0.8759
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.07912  Acc : 1.0
	 Saved at 1-epoch
	 Epoch : 1 | Train Loss : 0.1576 | Train Acc : 0.9424
	 Epoch : 1 | Valid Loss : 0.2522 | Valid Acc : 0.8993
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.01315  Acc : 1.0
	 Epoch : 2 | Train Loss : 0.04567 | Train Acc : 0.9851
	 Epoch : 2 | Valid Loss : 0.3739 | Valid Acc : 0.8913
[Train] Epoch :  3 [20010 / 20010 (100.0%)]  Loss: 0.001646  Acc : 1.0
	 Epoch : 3 | Train Loss : 0.01101 | Train Acc : 0.9968
	 Epoch : 3 | Valid Loss : 0.4561 | Valid Acc : 0.8895
[Train] Epoch :  4 [20010 / 20010 (100.0%)]  Loss: 0.006299  Acc : 1.0
	 Epoch : 4 | Train Loss : 0.006686 | Train Acc : 0.9979
	 Epoch : 4 

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

Test Loss : 0.278 | Test Acc : 0.8868


## Inference

In [ ]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
model.load_state_dict(torch.load(f"./{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}.pt"))

<All keys matched successfully>

In [ ]:
def predict_sentiment(model, sentence):
    model.eval()
    indexed = TEXT.numericalize(TEXT.pad([TEXT.tokenize(PreProcessingText(sentence))]))
    input_data = torch.LongTensor(indexed).to(device)
    prediction = torch.sigmoid(model(input_data))
    return prediction.item()

In [ ]:
test_sentence = 'this movie is FUN'
predict_sentiment(model = model, sentence = test_sentence)

0.9115353226661682